#### Import statements

In [1]:
import torch
import os
import shutil

#### Load the drive folder containing all required files

In [2]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# access the drive folder containing everything we need
%cd /content/drive/My Drive/Colab environments/Risiko! DL

# check that we are in the desired folder
%ls

Mounted at /content/drive
/content/drive/My Drive/Colab environments/Risiko! DL
 3D_models/
 backgrounds/
 coco_risiko.yaml
 custom_complete_yolo.yaml
 custom_yolo.yaml
 datasets/
 pre_trained_weights/
 real_images/
'Risiko!_Synthetic_Dataset_Creator.ipynb'
'Risiko! Test.ipynb'
 Split_train_test_val.ipynb
 synthetic_dataset/
 synthetic_images/
 Tanks_flags_detection_training.ipynb
 Test_detection.ipynb
 test_example.txt
 test.txt


#### Clone the GitHub repository yolov5 and install requirements

In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15814, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15814 (delta 10), reused 25 (delta 2), pack-reused 15768
Receiving objects: 100% (15814/15814), 14.64 MiB | 14.79 MiB/s, done.
Resolving deltas: 100% (10819/10819), done.
/content/drive/My Drive/Colab environments/Risiko! DL/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.5/627.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


## Load pre-trained YoloV5 Model

In [4]:
# set the device
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# number of classes: 6 tanks + 6 flags = 12 total classes
NUM_CLASSES = 12

# load the pre-trained model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, autoshape=False, force_reload=True, classes=NUM_CLASSES)


# save the model to the desired device
model.to(device)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-7-12 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Overriding model.yaml nc=80 with nc=12

                 from  n    params  module                                  arguments                     
  0                -1

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
     

## Training of the model only on synthetic images

#### Move the $.yaml$ file inside the $yolov5$ folder

In [5]:
# current path is $yolov5 folder
current_path = os.getcwd()
# name of the file containing training specifications
filename = "custom_yolo.yaml"
# source path of the file
file_path = os.path.join(current_path, "..", filename)
# path to the folder where the file has to be moved
dest_path = os.path.join(current_path, filename)
# copy the file in $file_path to $dest_path
shutil.copy(file_path, dest_path)

'/content/drive/My Drive/Colab environments/Risiko! DL/yolov5/custom_yolo.yaml'

#### Fine-tune the model

In [6]:
!python train.py --img 640 --epochs 500 --cache RAM --data custom_yolo.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=custom_yolo.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=RAM, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/content/drive/My': No such file or directory
YOLOv5 🚀 2023-7-12 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=

## Training of the model on both synthetic and real images

#### Move the $.yaml$ file inside the $yolov5$ folder

In [7]:
# current path is $yolov5 folder
current_path = os.getcwd()
# name of the file containing training specifications
filename = "custom_complete_yolo.yaml"
# source path of the file
file_path = os.path.join(current_path, "..", filename)
# path to the folder where the file has to be moved
dest_path = os.path.join(current_path, filename)
# copy the file in $file_path to $dest_path
shutil.copy(file_path, dest_path)

'/content/drive/MyDrive/Colab environments/Risiko! DL/yolov5/custom_complete_yolo.yaml'

#### Fine-tune the model

In [8]:
!python train.py --img 640 --epochs 500 --cache RAM --data custom_complete_yolo.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=custom_complete_yolo.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=RAM, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/content/drive/MyDrive/Colab': No such file or directory
YOLOv5 🚀 2023-7-12 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0

## Store the weights and the results of both training experiments
- $only\_synthetic$ will store the results of the training on only synthetic images;
- $synthetic\_and\_real$ will store the results of the training on both synthetic and real images.

In [9]:
# path to the folder where to store the results of training
results_dir_path = os.path.join(current_path, "..", "training_results")

# check if the folder exists and remove it in case it exists
if os.path.exists(results_dir_path):
    shutil.rmtree(results_dir_path)

# source path to the folder containing training results
folder_path = os.path.join(current_path, "runs", "train")

# copy the folder
shutil.copytree(folder_path, results_dir_path)

# rename the folder related to the training with only synthetic images
old_folder_path = os.path.join(results_dir_path, "exp")
new_folder_path = os.path.join(results_dir_path, "only_synthetic")
os.rename(old_folder_path, new_folder_path)

# rename the folder related to the training with all images
old_folder_path = os.path.join(results_dir_path, "exp2")
new_folder_path = os.path.join(results_dir_path, "synthetic_and_real")
os.rename(old_folder_path, new_folder_path)